In [1]:
import torch
import math
import numpy as np

dtype = torch.double
device_id = "cuda"

In [2]:
def generate_data(N, M, D, dtype=torch.double, device="cuda"):
    # sample D-dimensional array of random Tchebychev points in [-1,1]^D
    samples = math.pi * (
        torch.rand(N, D, dtype=dtype, device=device, requires_grad=False) - 0.5
    )
    samples = samples.tan()
    samples = samples / torch.sqrt(samples**2 + 1)

    # define D-dimensional array of indices in [0, R]^D based on hyperbolic cross density
    def hyp_cross(dim, R):
        if dim == 1:
            return np.arange(0, R + 1, dtype=np.int32).reshape(-1, 1)
        out = []
        for k in range(0, R + 1):
            prod_val = max(1, abs(k))
            R_reduced = R // prod_val
            sub_result = hyp_cross(dim - 1, R_reduced)
            extended = np.empty((len(sub_result), dim), dtype=np.int32)
            extended[:, 0] = k  # First dimension value (broadcast to all rows)
            extended[:, 1:] = sub_result  # Remaining dimensions from recursion
            out.append(extended)
        return np.vstack(out)

    # create hyperbolic cross indices
    indices= torch.from_numpy(hyp_cross(D, M)).to(dtype=torch.double, device=device)

    # function in H^3/2 with known Fourier coefficients
    def fun2(x):
        neg_mask = x <= 0
        out = torch.where(
            neg_mask,
            -(x**2) / 4 - x / 2 + 0.5,
            x**2 / 8 - x / 2 + 0.5,
        )
        out *= math.sqrt(1024 / (367 - 256 / math.pi))
        return out.prod(dim=1, keepdim=True)

    # Tchebychev coefficients
    def fun2_tchebychev_coeffs(indices):
        # Initialize output
        out = torch.zeros_like(indices, dtype=torch.double)

        # k = 0
        mask0 = indices == 0
        out = out + 15 / math.sqrt((367 - 256 / math.pi)) * mask0.double()

        # k = 1
        mask1 = indices == 1
        out = (
            out
            - 8
            / math.pi
            * math.sqrt(2 / (367 - 256 / math.pi))
            * (math.pi - 1)
            * mask1.double()
        )

        # k = 2
        mask2 = indices == 2
        out = out - 1 / math.sqrt((2 * 367 - 512 / math.pi)) * mask2.double()

        # k >= 3: Use exact formula without clamp
        mask_large = indices >= 3
        k_large = indices[mask_large]

        # Compute sin(π*k/2) exactly: alternates 0, 1, 0, -1, 0, 1, ...
        k_mod_4 = k_large % 4
        sin_vals = torch.where(
            k_mod_4 == 1,
            torch.ones_like(k_large),
            torch.where(
                k_mod_4 == 3, -torch.ones_like(k_large), torch.zeros_like(k_large)
            ),
        )

        coeff_large = (
            -24
            / math.pi
            * math.sqrt(2 / (367 - 256 / math.pi))
            * sin_vals
            / (k_large * (k_large**2 - 4))
        )

        out[mask_large] = coeff_large

        return torch.prod(out, dim=1, keepdim=True)

    # estimate norm and truncation error
    coeffs_gt = fun2_tchebychev_coeffs(indices)
    trunc_error = torch.sqrt(1 - torch.sum((coeffs_gt) ** 2))

    # convert k and x_gt to given dtype
    indices = indices.to(dtype=dtype)
    coeffs_gt = coeffs_gt.to(dtype=dtype)

    # create vector with normalized function values
    values = fun2(samples)

    # Optionally print info
    print("Number of indices in hyperbolic cross:", indices.size(0))
    print("Truncation error (double precision):", trunc_error.item())

    return indices, samples, values, coeffs_gt, trunc_error

In [3]:
# Here we generate the data
indices, samples, values, coeffs_gt, trunc_error = generate_data(
    15_000, 1000, 6, device=device_id, dtype=dtype
)

ind_array = [250, 500, 1000, 2000, 4000, 8000, 16000, 32000, 64000, 128000, 256000]
coeffs_gt = coeffs_gt.flatten()
sol_trunc = torch.zeros_like(coeffs_gt)
norm_out = torch.zeros(11)
i = 0
for k in ind_array:
    _, ind_trunc = torch.topk(torch.abs(coeffs_gt), k)
    sol_trunc[ind_trunc] = coeffs_gt[ind_trunc]
    norm_out[i] = torch.norm(coeffs_gt - sol_trunc)
    i = i + 1
torch.set_printoptions(precision=12, sci_mode=False)
print(norm_out)

Number of indices in hyperbolic cross: 4172784
Truncation error (double precision): 6.322027276634104e-08
tensor([    0.045129168779,     0.016127292067,     0.007196373306,
            0.002290656557,     0.000822020462,     0.000244717754,
            0.000079551020,     0.000023509072,     0.000007037104,
            0.000002032289,     0.000000578222])
